# Loading OpenAI API key

In [15]:
from openai import OpenAI
import openai
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
with open('../sample_data/token.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [17]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [18]:
client = OpenAI()

In [19]:
# text-embedding-ada 불러서 임베딩 가져오는 펑션
def get_embeddings(text):
  return client.embeddings.create(
    model="text-embedding-ada-002",
    input=text,
    encoding_format="float"
  ).data[0].embedding

# Creating Mini VectorDB

## Creating Embedding concepts

In [33]:
# 임베딩을 만들어봅시다!
no_mon_embed      = get_embeddings("돈이 없다")
timetravel_embed  = get_embeddings("회귀")
bodysnat_embed    = get_embeddings("빙의")
reincarn_embed    = get_embeddings("환생")

In [34]:
solutions = {
    "돈이 없다": {
        "embeddings": no_mon_embed,
        "solutions": ["돈 있는 캐릭터가 된다!"]
    },
    "회귀": {
        "embeddings": timetravel_embed,
        "solutions": ["시간을 거슬러 되돌아 감!"]
    },
    "빙의": {
        "embeddings": bodysnat_embed,
        "solutions": ["소설속 캐릭터로 빙의하자!"]
    },
    "환생": {
        "embeddings": reincarn_embed,
        "solutions": ["판타지 소설의 캐릭터로 다시 태어나자!"]
    }
}

### Process : Find similar answer & category from mini-vectorDB 

In [22]:
def get_highest_match(problem_embed):
    # 사용자의 문제를 받고, 우리의 미니 DB 에서 제일 가까운 솔루션을 찾아낼 것입니다!
    results = []
    for key, value in solutions.items():
        sim = cosine_similarity([problem_embed], [value['embeddings']])[0][0]
        results.append((key, sim, value['solutions']))

    # 유사도로 정리
    results.sort(key=lambda x: x[1], reverse=True)

    # 제일 유사한 하나만 가져오기
    return results[0]

In [23]:
import random

In [24]:
def get_solution(problem) -> str:
    # 사용자의 문제를 임베딩으로 바꿈
    problem_embed = get_embeddings(problem)

    # 제일 가까운 카테고리의 솔루션을 가지고 옴
    solution_found = get_highest_match(problem_embed)[2][0]
    return solution_found

# Prompt Engineering

In [26]:
def get_problem(user_query):
  # 사용자의 질문을 듣고 카테고리를 선별함
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """사용자의 질문을 듣고 그 문제를 해결할 수 있는 방법을 다음 다섯가지 중에서 선택해줘:
            '돈이 없다', '회귀', '빙의', '환생'
            이 중에 선택할 옵션이 없으면 그냥 아무 대답도 하지 마.
            """
            },
            {"role": "user", "content": user_query},
        ]
    ).choices[0].message.content

In [28]:
def get_detailed_solution(user_query):
  # 모든 자료가 모였으니 이제 마지막 답변을 생성함
  colors = ["빨강", "검정", "금색", "은색", "회색", "파랑", "녹색", "보라색", "자주색"]
  solution    = get_solution(get_problem(user_query))
  hair_color  = random.choice(colors)
  eye_color   = random.choice(colors)
  job         = random.choice(["마법사", "왕족", "황족", "기사", "사제", "상단주"])
  family      = random.choice(["황족", "왕족", "공녀", "평민", "하녀", "백작가", "노예", "공작가", "야만족"])
  
  # Prompt Template
  story_template = f"""
            이번 사용자의 불평은 {user_query} 이며, 이것에 대한 기본적인 해결법은 {solution}이다.
            다음의 설정을 바탕으로 사용자에게 그럴듯한 판타지 스토리를 제안해라.
            이번 시나리오에서 사용할 설정은:
            주인공의 머리색: {hair_color}
            주인공의 눈 색: {eye_color}
            주인공의 신분: {family}
            주인공의 직업: {job}
            기본적인 해결법: {solution}
            이 캐릭터가 사는 가상의 나라 이름을 정하고 위의 설정을 바탕으로 해서 500자 내외의 시나리오를 만들어낸다.
            
            판타지 설정:"""
  
  print(f"프롬프트: {story_template}")
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """너는 퐌타스틱 스토리 작가 AI이다.
            너의 역할은 주변에서 흔히 일어날 수 있는 문제를 가진 사람의 불평을 기본으로,
            판타지스러운 설정을 만들어내는 것이다."""
            },
            {"role": "user", "content": user_query},
            {"role": "system", "content": story_template},
        ]
    ).choices[0].message.content

In [29]:
# 돌려봅시다!
get_detailed_solution("사는게 참 힘든데 내가 참 고등학교로 돌아간다면은 공부를 좀 더 열심히 할 것 같아")

프롬프트: 
            이번 사용자의 불평은 사는게 참 힘든데 내가 참 고등학교로 돌아간다면은 공부를 좀 더 열심히 할 것 같아 이며, 이것에 대한 기본적인 해결법은 시간을 거슬러 되돌아 감!이다.
            다음의 설정을 바탕으로 사용자에게 그럴듯한 판타지 스토리를 제안해라.
            이번 시나리오에서 사용할 설정은:
            주인공의 머리색: 자주색
            주인공의 눈 색: 보라색
            주인공의 신분: 평민
            주인공의 직업: 마법사
            기본적인 해결법: 시간을 거슬러 되돌아 감!
            이 캐릭터가 사는 가상의 나라 이름을 정하고 위의 설정을 바탕으로 해서 500자 내외의 시나리오를 만들어낸다.
            
            판타지 설정:


'"자주색 머리와 보라색 눈을 가진 평민 마법사, 레오는 \'테라노아\'라는 이름의 환상적인 나라에서 살고 있었습니다. 그는 일상의 고단함에 지쳐, 고등학교 시절로 돌아가서 공부를 더 열심히 하고 싶었습니다. 그런데 그의 마법사로서의 능력은 시간을 조작하는 것이었습니다.\n\n레오는 자신의 마법을 이용해 과거로 돌아가기로 결심했습니다. 그는 자신의 마법서를 펼쳐, 시간을 거슬러 올라가는 주문을 외웠습니다. 그리고 그는 자신의 보라색 눈을 감고, 마법의 힘을 집중했습니다. 그의 주위에는 시간의 흐름이 뒤틀리는 것 같은 기묘한 현상이 일어났습니다.\n\n그의 눈이 떠졌을 때, 그는 자신이 고등학교 교실에 앉아 있는 것을 발견했습니다. 그는 놀랍게도 과거로 돌아왔습니다. 그는 이 기회를 허비하지 않고, 공부에 몰두하기로 결심했습니다. 그는 과거의 자신이 놓쳤던 기회를 잡기 위해, 더 열심히 공부했습니다.\n\n그의 노력 덕분에, 그는 고등학교를 졸업하고, 더 나은 미래를 만들어 나갔습니다. 그는 이 경험을 통해, 공부의 중요성을 깨닫게 되었습니다. 그는 이제, 자신의 미래를 위해 더 열심히 살아가기로 결심했습니다."'

# Generating Image

In [30]:
image_prompt = '자주색 머리와 보라색 눈을 가진 평민 마법사, 레오는 테라노아라는 이름의 환상적인 나라에서 살고 있었습니다. 그는 일상의 고단함에 지쳐, 고등학교 시절로 돌아가서 공부를 더 열심히 하고 싶었습니다. 그런데 그의 마법사로서의 능력은 시간을 조작하는 것이었습니다. 레오는 자신의 마법을 이용해 과거로 돌아가기로 결심했습니다. 그는 자신의 마법서를 펼쳐, 시간을 거슬러 올라가는 주문을 외웠습니다. 그리고 그는 자신의 보라색 눈을 감고, 마법의 힘을 집중했습니다. 그의 주위에는 시간의 흐름이 뒤틀리는 것 같은 기묘한 현상이 일어났습니다.그의 눈이 떠졌을 때, 그는 자신이 고등학교 교실에 앉아 있는 것을 발견했습니다. 그는 놀랍게도 과거로 돌아왔습니다. 그는 이 기회를 허비하지 않고, 공부에 몰두하기로 결심했습니다. 그는 과거의 자신이 놓쳤던 기회를 잡기 위해, 더 열심히 공부했습니다.그의 노력 덕분에, 그는 고등학교를 졸업하고, 더 나은 미래를 만들어 나갔습니다. 그는 이 경험을 통해, 공부의 중요성을 깨닫게 되었습니다. 그는 이제, 자신의 미래를 위해 더 열심히 살아가기로 결심했습니다.'

In [31]:
response = client.images.generate(
  model="dall-e-3",
  prompt=image_prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

In [32]:
from IPython.display import display, HTML
html = f'<img src="{image_url}" width="600">' # You can adjust the width as needed
display(HTML(html))